## Gesture Recognition ##

In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 100kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
## Read google drive file
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Importing the required libraries

In [ ]:
import numpy as np
import os
from scipy.misc import imread, imresize
import imageio
import datetime
import keras
import os


We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

Importinng the csv files of training and validation sets

In [ ]:
train_doc = np.random.permutation(open('/content/gdrive/MyDrive/Files/DS/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/gdrive/MyDrive/Files/DS/Project_data/val.csv').readlines())
batch_size = 32  #experiment with the batch size

Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with img_idx, y,z and normalization such that you get high accuracy.

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [ ]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/gdrive/MyDrive/Files/DS/Project_data/train'
val_path = '/content/gdrive/MyDrive/Files/DS/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


Model

Here you make the model using different functionalities that Keras provides. Remember to use Conv3D and MaxPooling3D and not Conv2D and Maxpooling2D for a 3D convolution model. You would want to use TimeDistributed while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import LeakyReLU

#write your model here
input_shape = (30, 120, 120, 3)
model= Sequential()
model.add(Conv3D(8,kernel_size =(3,3,3),input_shape=(30,120,120,3),padding = 'same'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=.001))

# Layer_1

model.add(Conv3D(16,(3,3,3),padding ='same'))
model.add(LeakyReLU(alpha=.001))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))

# Layer_2

model.add(Conv3D(32,(3,3,3),padding ='same'))
model.add(LeakyReLU(alpha=.001))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))

# Layer_3

model.add(Conv3D(64,(2,2,2),padding ='same'))
model.add(LeakyReLU(alpha=.001))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))

# Layer_4 till layer 3 i was getting accuaracy [loss: 1.6099 - categorical_accuracy: 0.1830 - val_loss: 1.6149 - val_categorical_accuracy: 0.1250] 

model.add(Conv3D(128,(2,2,2),padding ='same'))
model.add(LeakyReLU(alpha=.001))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))

# Flatten layer 

model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))

#Softmax layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to compile the model. When you print the summary of the model, you'll see the total number of parameters you have to train.

In [ ]:
#optimiser = Keras.optimizers.Adam(lr=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 120, 120, 8)   32        
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 30, 120, 120, 8)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 30, 120, 120, 16)  3472      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 60, 60, 16)    0

Let us create the train_generator and the val_generator which will be used in .fit_generator.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the REducelronplateau code here
LR = ReduceLROnPlateau(monitor = "val_loss", 
                       factor = 0.1, 
                       patience = 10,   
                       verbose = 0, 
                       mode = "auto", 
                       epsilon = 1e-04, 
                       cooldown = 0,
                       min_lr = 0)

callbacks_list = [checkpoint, LR]

The steps_per_epoch and validation_steps are used by fit_generator to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
print('steps_per_epoch: ',steps_per_epoch)
print('validation_steps: ',validation_steps)

steps_per_epoch:  21
validation_steps:  4


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit(train_generator, 
          steps_per_epoch=steps_per_epoch,                     
          epochs=num_epochs,                     
          verbose=1,                     
          callbacks=callbacks_list,                     
          validation_data=val_generator,                     
          validation_steps=validation_steps,                     
          class_weight=None,                     
          workers=1,                     
          initial_epoch=0)

Source path =  /content/gdrive/MyDrive/Files/DS/Project_data/train ; batch size = 32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


Epoch 1/30
19/21 [==========================>...] - ETA: 6:15 - loss: 8.6023 - categorical_accuracy: 0.3072Batch:  21 Index: 32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


21/21 [==============================] - ETA: 0s - loss: 8.3411 - categorical_accuracy: 0.3088  Source path =  /content/gdrive/MyDrive/Files/DS/Project_data/val ; batch size = 32
Batch:  4 Index: 32
21/21 [==============================] - 4276s 212s/step - loss: 8.2247 - categorical_accuracy: 0.3097 - val_loss: 1.4490 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2021-04-0311_28_19.104657/model-00001-5.78038-0.32881-1.44901-0.32000.h5
Epoch 2/30
21/21 [==============================] - 168s 8s/step - loss: 1.9760 - categorical_accuracy: 0.4741 - val_loss: 1.7376 - val_categorical_accuracy: 0.3750

Epoch 00002: saving model to model_init_2021-04-0311_28_19.104657/model-00002-1.87535-0.47205-1.73755-0.37500.h5
Epoch 3/30
21/21 [==============================] - 84s 4s/step - loss: 1.4240 - categorical_accuracy: 0.5486 - val_loss: 3.5625 - val_categorical_accuracy: 0.1875

Epoch 00003: saving model to model_init_2021-04-0311_28_19.104657/model-00003-1.46409-

Observation: The final accuracy that we see for 30 epochs is loss: 0.1350 - categorical_accuracy: 0.9310 - val_loss: 0.2832 - val_categorical_accuracy: 0.8750, which could be greatly improved as we increase the epochs to 50,100...


Note: All the details of file coding, parameter used and working will be explained in the write up doc file 

In [ ]:
model.save('model.h5')

The model file is downloaded and will be included in the attachments